# Fase 3: Trabalho Prático de Mineração de Dados - Conclusão e Análise Comparativa

**Alunos:** Gilmar Junio, Fabrício Chaves

Este notebook documenta a Fase 3 do Trabalho Prático de Mineração de Dados. O objetivo é consolidar as análises de padrões frequentes em dados de e-commerce, realizar análises segmentadas propostas, refinar a avaliação e apresentar uma análise comparativa do trabalho desenvolvido com auxílio do LLM (Fase 2) versus as conclusões e refinamentos do grupo (Fase 3).

## 1. Business Understanding (Entendimento do Negócio)

(Esta seção pode ser copiada da Fase 2, pois o contexto de negócio permanece o mesmo)

**Contexto:** Análise de transações de vendas de um e-commerce do Reino Unido.

**Relevância:**
*   Compreender o comportamento de compra dos clientes.
*   Melhorar estratégias de *cross-selling* e *up-selling*.
*   Otimizar estoque e promoções com base em produtos frequentemente comprados juntos.

**Objetivo Principal:** Identificar itens frequentemente comprados juntos para embasar recomendações de produtos e estratégias de marketing.

**Objetivos Específicos:**
*   Encontrar *itemsets* frequentes (conjuntos de produtos que aparecem juntos em transações).
*   Extrair regras de associação (ex.: "Se compra X, então compra Y").
*   Analisar possíveis diferenças nos padrões por país ou tipo de cliente (se viável).

**Critérios de Sucesso (Negócio):**
*   Identificar regras de associação acionáveis.
*   Gerar visualizações claras dos padrões encontrados.
*   Propor ações práticas (ex.: promoções combinadas, layout de site).

**Objetivos da Mineração de Dados:**
*   Aplicar algoritmos como Apriori ou FP-Growth para encontrar *itemsets* frequentes.
*   Gerar regras de associação com métricas de suporte, confiança e *lift*.
*   Filtrar regras para focar nas mais relevantes (ex: `lift > 1`).

**Critérios de Sucesso (Mineração):**
*   Gerar regras com `min_support` e `min_confidence` definidos (ex: 0.01 e 0.5, conforme proposta).
*   Avaliar a qualidade das regras encontradas usando as métricas.

---

## 2. Data Understanding & Data Preparation (Entendimento e Preparação dos Dados)

Nesta etapa, carregamos os dados originais e aplicamos os passos de limpeza e transformação consolidados da Fase 2.

In [1]:
# Importar bibliotecas necessárias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import fpgrowth, association_rules

# Configurações de visualização (opcional)
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
pd.set_option('display.max_colwidth', None)

In [2]:
# Carregar o dataset original
file_path = 'Sales Transaction v.4a.csv' # Ajuste se o nome/caminho for diferente
try:
    df_original = pd.read_csv(file_path)
except UnicodeDecodeError:
    df_original = pd.read_csv(file_path, encoding='latin1')

print(f'Dataset original carregado com {df_original.shape[0]} linhas e {df_original.shape[1]} colunas.')

# Aplicar limpeza consolidada (baseada nas decisões da Fase 2)
df_cleaned = df_original.copy()

# 1. Remover linhas com CustomerNo nulo
df_cleaned.dropna(subset=['CustomerNo'], inplace=True)

# 2. Converter Date para datetime
try:
    df_cleaned['Date'] = pd.to_datetime(df_cleaned['Date'])
except ValueError:
    df_cleaned['Date'] = pd.to_datetime(df_cleaned['Date'], format='%m/%d/%Y')

# 3. Remover linhas com Quantity <= 0
df_cleaned = df_cleaned[df_cleaned['Quantity'] > 0]

# 4. Remover linhas com Price <= 0 (verificação de segurança, embora na Fase 2 não removeu nenhuma)
df_cleaned = df_cleaned[df_cleaned['Price'] > 0]

# 5. Remover transações canceladas (TransactionNo começando com 'C')
df_cleaned['TransactionNo'] = df_cleaned['TransactionNo'].astype(str)
df_cleaned = df_cleaned[~df_cleaned['TransactionNo'].str.startswith('C')]

# 6. Converter CustomerNo para inteiro
df_cleaned['CustomerNo'] = df_cleaned['CustomerNo'].astype(int)

# 7. Remover duplicatas
df_cleaned.drop_duplicates(inplace=True)

# 8. Limpar espaços extras em ProductName
df_cleaned['ProductName'] = df_cleaned['ProductName'].str.strip()

print(f'Dataset após limpeza: {df_cleaned.shape[0]} linhas e {df_cleaned.shape[1]} colunas.')
df_cleaned.info()

Dataset original carregado com 536350 linhas e 8 colunas.
Dataset após limpeza: 522601 linhas e 8 colunas.
<class 'pandas.core.frame.DataFrame'>
Index: 522601 entries, 0 to 536324
Data columns (total 8 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   TransactionNo  522601 non-null  object        
 1   Date           522601 non-null  datetime64[ns]
 2   ProductNo      522601 non-null  object        
 3   ProductName    522601 non-null  object        
 4   Price          522601 non-null  float64       
 5   Quantity       522601 non-null  int64         
 6   CustomerNo     522601 non-null  int32         
 7   Country        522601 non-null  object        
dtypes: datetime64[ns](1), float64(1), int32(1), int64(1), object(4)
memory usage: 33.9+ MB


### 2.1. Transformação para Formato Transacional (Dataset Completo)

Agrupamos os itens por transação para o dataset limpo completo.

In [3]:
# Agrupar por TransactionNo e listar os ProductName para o dataset completo
transactions_series_all = df_cleaned.groupby('TransactionNo')['ProductName'].apply(list)
transactions_list_all = transactions_series_all.tolist()

print(f'Número de transações únicas (dataset completo): {len(transactions_list_all)}')
if len(transactions_list_all) > 1:
    print('\nExemplo das primeiras 2 listas de transações (dataset completo):')
    for i in range(min(2, len(transactions_list_all))):
        print(f"Transação {i+1}: {transactions_list_all[i]}")
else:
    print("Não há transações suficientes para mostrar exemplos.")

Número de transações únicas (dataset completo): 19789

Exemplo das primeiras 2 listas de transações (dataset completo):
Transação 1: ['Cream Hanging Heart T-Light Holder', 'White Moroccan Metal Lantern', 'Cream Cupid Hearts Coat Hanger', 'Knitted Union Flag Hot Water Bottle', 'Red Woolly Hottie White Heart', 'Set 7 Babushka Nesting Boxes', 'Glass Star Frosted T-Light Holder']
Transação 2: ['Hand Warmer Union Jack', 'Hand Warmer Red Retrospot']


## 3. Modeling (Modelagem - Análise Geral)

Aplicamos o algoritmo FP-Growth ao dataset completo limpo para encontrar padrões frequentes e regras de associação. Estes resultados serão predominantemente influenciados pelas transações do Reino Unido, dada a sua proporção nos dados.

In [4]:
# Codificar os dados completos (One-Hot Encoding)
te_all = TransactionEncoder()
te_ary_all = te_all.fit(transactions_list_all).transform(transactions_list_all)
df_encoded_all = pd.DataFrame(te_ary_all, columns=te_all.columns_)

print(f"Dimensões do DataFrame codificado (completo): {df_encoded_all.shape}")

# Aplicar o FP-Growth
min_support_threshold_all = 0.01
frequent_itemsets_all = fpgrowth(df_encoded_all, min_support=min_support_threshold_all, use_colnames=True)
frequent_itemsets_all = frequent_itemsets_all.sort_values(by='support', ascending=False)

print(f"\nEncontrados {len(frequent_itemsets_all)} itemsets frequentes (dataset completo) com suporte >= {min_support_threshold_all}")
print("\nTop 10 itemsets frequentes (dataset completo):")
print(frequent_itemsets_all.head(10))

# Gerar Regras de Associação
min_confidence_threshold_all = 0.5
rules_all = association_rules(frequent_itemsets_all, metric="confidence", min_threshold=min_confidence_threshold_all)
rules_all = rules_all[rules_all['lift'] > 1]
rules_all = rules_all.sort_values(by=['lift', 'confidence'], ascending=[False, False])

print(f"\nEncontradas {len(rules_all)} regras de associação (dataset completo) com confiança >= {min_confidence_threshold_all} e lift > 1")
print("\nTop 10 regras de associação (dataset completo, ordenadas por lift e confiança):")
print(rules_all.head(10)[['antecedents', 'consequents', 'support', 'confidence', 'lift']])

Dimensões do DataFrame codificado (completo): (19789, 3753)

Encontrados 2029 itemsets frequentes (dataset completo) com suporte >= 0.01

Top 10 itemsets frequentes (dataset completo):
      support                              itemsets
0    0.114660  (Cream Hanging Heart T-Light Holder)
96   0.105715             (Jumbo Bag Red Retrospot)
313  0.100510            (Regency Cakestand 3 Tier)
615  0.085249                       (Party Bunting)
43   0.079185             (Lunch Bag Red Retrospot)
8    0.073627       (Assorted Colour Bird Ornament)
109  0.070342                      (Popcorn Holder)
647  0.069988    (Set Of 3 Cake Tins Pantry Design)
44   0.066805     (Pack Of 72 Retrospot Cake Cases)
178  0.064986               (Lunch Bag Suki Design)

Encontradas 963 regras de associação (dataset completo) com confiança >= 0.5 e lift > 1

Top 10 regras de associação (dataset completo, ordenadas por lift e confiança):
                                     antecedents  \
859  (Herb Marker Ros

---
## 4. Modeling (Modelagem - Análise Segmentada por País)

Conforme o objetivo específico da proposta, analisamos se existem padrões de compra diferentes em outros países. Dada a distribuição dos dados, focaremos na Alemanha e, se viável, na França.

### 4.1. Análise para a Alemanha

In [5]:
# Filtrar dados para a Alemanha
df_germany = df_cleaned[df_cleaned['Country'] == 'Germany']
print(f"Número de registros para a Alemanha: {len(df_germany)}")

# Transformar para formato transacional (Alemanha)
transactions_series_germany = df_germany.groupby('TransactionNo')['ProductName'].apply(list)
transactions_list_germany = transactions_series_germany.tolist()
print(f"Número de transações únicas (Alemanha): {len(transactions_list_germany)}")

if len(transactions_list_germany) > 10: # Checar se há um número mínimo razoável de transações
    te_germany = TransactionEncoder()
    te_ary_germany = te_germany.fit(transactions_list_germany).transform(transactions_list_germany)
    df_encoded_germany = pd.DataFrame(te_ary_germany, columns=te_germany.columns_)
    print(f"Dimensões do DataFrame codificado (Alemanha): {df_encoded_germany.shape}")

    # Aplicar FP-Growth para Alemanha
    # Pode ser necessário um min_support menor devido ao volume de dados
    min_support_germany = 0.01 # Comece com 0.01, ajuste se necessário (ex: 0.02, 0.03 ou até mais se gerar muitas regras)
    
    # Tentar com min_support inicial, se gerar poucas/muitas regras, ajustar
    frequent_itemsets_germany = fpgrowth(df_encoded_germany, min_support=min_support_germany, use_colnames=True)
    
    # Ajuste dinâmico do min_support se poucas regras forem encontradas
    # Esta é uma heurística simples, pode precisar de mais refinamento
    if len(frequent_itemsets_germany) < 10 and len(frequent_itemsets_germany) > 0 : # Se muito poucas, tentar reduzir o suporte
        print(f"Poucos itemsets com suporte {min_support_germany}. Tentando com suporte menor...")
        min_support_germany = 0.005 
        frequent_itemsets_germany = fpgrowth(df_encoded_germany, min_support=min_support_germany, use_colnames=True)
    elif len(frequent_itemsets_germany) > 5000: # Se muitas, tentar aumentar o suporte
         print(f"Muitos itemsets com suporte {min_support_germany}. Tentando com suporte maior...")
         min_support_germany = 0.03
         frequent_itemsets_germany = fpgrowth(df_encoded_germany, min_support=min_support_germany, use_colnames=True)


    if not frequent_itemsets_germany.empty:
        frequent_itemsets_germany = frequent_itemsets_germany.sort_values(by='support', ascending=False)
        print(f"\nEncontrados {len(frequent_itemsets_germany)} itemsets frequentes (Alemanha) com suporte >= {min_support_germany}")
        print("\nTop 5 itemsets frequentes (Alemanha):")
        print(frequent_itemsets_germany.head(5))

        # Gerar Regras de Associação para Alemanha
        min_confidence_germany = 0.2 # Pode ser necessário ajustar a confiança também, talvez um pouco menor
        rules_germany = association_rules(frequent_itemsets_germany, metric="confidence", min_threshold=min_confidence_germany)
        
        if not rules_germany.empty:
            rules_germany = rules_germany[rules_germany['lift'] > 1]
            rules_germany = rules_germany.sort_values(by=['lift', 'confidence'], ascending=[False, False])
            print(f"\nEncontradas {len(rules_germany)} regras de associação (Alemanha) com confiança >= {min_confidence_germany} e lift > 1")
            print("\nTop 5 regras de associação (Alemanha, ordenadas por lift e confiança):")
            print(rules_germany.head(5)[['antecedents', 'consequents', 'support', 'confidence', 'lift']])
        else:
            print("Nenhuma regra de associação encontrada para a Alemanha com os critérios atuais.")
    else:
        print(f"Nenhum itemset frequente encontrado para a Alemanha com suporte >= {min_support_germany}.")
else:
    print("Número de transações para a Alemanha é muito baixo para uma análise significativa de padrões frequentes.")


Número de registros para a Alemanha: 10221
Número de transações únicas (Alemanha): 453
Dimensões do DataFrame codificado (Alemanha): (453, 2069)
Muitos itemsets com suporte 0.01. Tentando com suporte maior...

Encontrados 260 itemsets frequentes (Alemanha) com suporte >= 0.03

Top 5 itemsets frequentes (Alemanha):
     support                              itemsets
19  0.240618  (Round Snack Boxes Set Of4 Woodland)
20  0.154525   (Round Snack Boxes Set Of 4 Fruits)
21  0.141280            (Regency Cakestand 3 Tier)
40  0.136865    (Plasters In Tin Woodland Animals)
22  0.132450              (Woodland Charlotte Bag)

Encontradas 185 regras de associação (Alemanha) com confiança >= 0.2 e lift > 1

Top 5 regras de associação (Alemanha, ordenadas por lift e confiança):
                        antecedents                     consequents   support  \
105        (Spaceboy Childrens Cup)       (Spaceboy Childrens Bowl)  0.035320   
106       (Spaceboy Childrens Bowl)        (Spaceboy Childrens 

### 4.2. Análise para a França

In [6]:
# Filtrar dados para a França
df_france = df_cleaned[df_cleaned['Country'] == 'France']
print(f"Número de registros para a França: {len(df_france)}")

# Transformar para formato transacional (França)
transactions_series_france = df_france.groupby('TransactionNo')['ProductName'].apply(list)
transactions_list_france = transactions_series_france.tolist()
print(f"Número de transações únicas (França): {len(transactions_list_france)}")

if len(transactions_list_france) > 10: # Checar se há um número mínimo razoável de transações
    te_france = TransactionEncoder()
    te_ary_france = te_france.fit(transactions_list_france).transform(transactions_list_france)
    df_encoded_france = pd.DataFrame(te_ary_france, columns=te_france.columns_)
    print(f"Dimensões do DataFrame codificado (França): {df_encoded_france.shape}")

    # Aplicar FP-Growth para França
    min_support_france = 0.02 # Começar com um valor um pouco maior que o da Alemanha, ajustar se necessário
    
    frequent_itemsets_france = fpgrowth(df_encoded_france, min_support=min_support_france, use_colnames=True)
    
    # Ajuste dinâmico do min_support
    if len(frequent_itemsets_france) < 5 and len(frequent_itemsets_france) > 0:
        print(f"Poucos itemsets com suporte {min_support_france}. Tentando com suporte menor...")
        min_support_france = 0.01
        frequent_itemsets_france = fpgrowth(df_encoded_france, min_support=min_support_france, use_colnames=True)
    elif len(frequent_itemsets_france) > 3000:
         print(f"Muitos itemsets com suporte {min_support_france}. Tentando com suporte maior...")
         min_support_france = 0.04
         frequent_itemsets_france = fpgrowth(df_encoded_france, min_support=min_support_france, use_colnames=True)

    if not frequent_itemsets_france.empty:
        frequent_itemsets_france = frequent_itemsets_france.sort_values(by='support', ascending=False)
        print(f"\nEncontrados {len(frequent_itemsets_france)} itemsets frequentes (França) com suporte >= {min_support_france}")
        print("\nTop 5 itemsets frequentes (França):")
        print(frequent_itemsets_france.head(5))

        # Gerar Regras de Associação para França
        min_confidence_france = 0.2 # Manter a confiança em 0.2 por enquanto
        rules_france = association_rules(frequent_itemsets_france, metric="confidence", min_threshold=min_confidence_france)
        
        if not rules_france.empty:
            rules_france = rules_france[rules_france['lift'] > 1]
            rules_france = rules_france.sort_values(by=['lift', 'confidence'], ascending=[False, False])
            print(f"\nEncontradas {len(rules_france)} regras de associação (França) com confiança >= {min_confidence_france} e lift > 1")
            print("\nTop 5 regras de associação (França, ordenadas por lift e confiança):")
            print(rules_france.head(5)[['antecedents', 'consequents', 'support', 'confidence', 'lift']])
        else:
            print("Nenhuma regra de associação encontrada para a França com os critérios atuais.")
    else:
        print(f"Nenhum itemset frequente encontrado para a França com suporte >= {min_support_france}.")
else:
    print("Número de transações para a França é muito baixo para uma análise significativa de padrões frequentes.")

Número de registros para a França: 10377
Número de transações únicas (França): 404
Dimensões do DataFrame codificado (França): (404, 2027)

Encontrados 1370 itemsets frequentes (França) com suporte >= 0.02

Top 5 itemsets frequentes (França):
      support                            itemsets
256  0.193069                (Rabbit Night Light)
0    0.190594     (Red Toadstool Led Night Light)
146  0.170792     (Plasters In Tin Circus Parade)
29   0.168317  (Plasters In Tin Woodland Animals)
18   0.163366           (Lunch Bag Red Retrospot)

Encontradas 3737 regras de associação (França) com confiança >= 0.2 e lift > 1

Top 5 regras de associação (França, ordenadas por lift e confiança):
                                              antecedents  \
2887                           (Set/6 Collage Paper Cups)   
2888                         (Set/6 Collage Paper Plates)   
2714                 (Small Dolly Mix Design Orange Bowl)   
2715                       (Small Marshmallows Pink Bowl)   
19

---
## 5. Evaluation (Avaliação Consolidada)

Nesta seção, avaliamos os resultados das análises geral e segmentada, conectamos os achados aos objetivos de negócio e discutimos as implicações.

### 5.1. Comparação dos Padrões Encontrados

*   **Análise Geral (Principalmente UK):**
    *   **Itemsets Frequentes Notáveis:** `Cream Hanging Heart T-Light Holder`, `Jumbo Bag Red Retrospot`, `Regency Cakestand 3 Tier`.
    *   **Regras Fortes:** Associações muito fortes entre `Herb Markers` (Thyme, Basil, Parsley, Rosemary, Mint) com altíssimo lift e confiança.
*   **Análise Segmentada - Alemanha:**
    *   **Itemsets Frequentes Notáveis:** `Round Snack Boxes Set Of4 Woodland`, `Round Snack Boxes Set Of 4 Fruits`, `Regency Cakestand 3 Tier`.
    *   **Regras Fortes:** Associações fortes entre itens infantis como `(Spaceboy Childrens Cup) <-> (Spaceboy Childrens Bowl)` e `(Childrens Cutlery Spaceboy) <-> (Childrens Cutlery Dolly Girl)`, e também entre designs de canecas.
*   **Análise Segmentada - França:**
    *   **Itemsets Frequentes Notáveis:** `Rabbit Night Light`, `Red Toadstool Led Night Light`, `Plasters In Tin Circus Parade`.
    *   **Regras Fortes:** Associações quase perfeitas (confiança 1.0 ou 0.9) para conjuntos de itens de festa como `(Set/6 Collage Paper Cups) <-> (Set/6 Collage Paper Plates)` e pequenos potes, além de combinações específicas de conjuntos infantis.

**Observações da Comparação:**
*   Os itens mais populares individualmente e os padrões de associação mais fortes variam consideravelmente entre os segmentos geográficos.
*   O `Regency Cakestand 3 Tier` parece ter uma popularidade consistente tanto no UK/Geral quanto na Alemanha.
*   A Alemanha e a França mostram fortes padrões de compra de "conjuntos" (itens infantis, itens de festa, designs coordenados).
*   O Reino Unido (análise geral) se destaca pelo forte cluster dos `Herb Markers`.

### 5.2. Visualizações Comparativas (Opcional Avançado)

(Poderíamos tentar plotar os top N itens/regras de cada país lado a lado, ou usar grafos comparativos, mas isso pode ser complexo para este escopo. Por ora, a análise descritiva acima é suficiente para a comparação.)

### 5.3. Conexão com Objetivos de Negócio e Proposta de Ações (Consolidado)

Com base nos padrões gerais e segmentados:

*   **Cross-selling/Up-selling e Bundling:**
    *   **Geral/UK:** Promover kits de "Herb Markers".
    *   **Alemanha:** Oferecer conjuntos "Spaceboy" (copo+tigela) e "Childrens Cutlery" (Spaceboy+Dolly Girl). Promover a compra de múltiplas canecas com designs diferentes.
    *   **França:** Criar bundles de itens de festa ("Kit Festa Collage"). Oferecer os pares de potes/tigelas identificados. Destacar os conjuntos infantis que são comprados de forma complementar.
    *   **Comum:** Para o `Regency Cakestand 3 Tier`, sugerir itens de chá ou confeitaria. Para `Lunch Bags`, sugerir lancheiras ou garrafas.
*   **Otimização de Estoque e Promoções:**
    *   **Layout:** Agrupar os itens identificados como conjuntos em cada país/região no site ou loja física.
    *   **Promoções:**
        *   UK: "Compre 3 marcadores de ervas, leve o 4º com desconto".
        *   Alemanha: "Leve o conjunto Spaceboy com X% de desconto".
        *   França: "Desconto na compra do conjunto de pratos e copos de festa".
    *   Considerar o suporte dos itens ao planejar o volume de promoções.

### 5.4. Avaliação dos Critérios de Sucesso (Consolidado)

*   **Critérios de Mineração:**
    *   Dataset completo: 2029 itemsets, 963 regras.
    *   Alemanha: 260 itemsets (suporte 0.03), 185 regras (confiança 0.2).
    *   França: 1370 itemsets (suporte 0.02), 3737 regras (confiança 0.2).
    *   Os parâmetros foram ajustados para os segmentos menores para obter um número razoável de regras, demonstrando flexibilidade na aplicação.
*   **Critérios de Negócio:**
    *   Foram identificadas regras acionáveis e específicas para diferentes mercados.
    *   As visualizações (da Fase 2 e as análises tabulares aqui) ajudam a entender os padrões.
    *   Ações práticas foram propostas com base nos achados.

### 5.5. Limitações e Próximos Passos

*   **Volume de Dados por Segmento:** Para países além de UK, Alemanha e França, o volume de transações é muito baixo para uma análise de regras de associação robusta com os mesmos parâmetros.
*   **Análise de "Tipo de Cliente":** Não foi realizada devido à falta de dados/critérios para segmentação de clientes no dataset.
*   **Outras Métricas:** Focamos em suporte, confiança e lift. Outras métricas poderiam trazer novas perspectivas.
*   **Próximos Passos:**
    *   Implementar e testar A/B as ações de negócio propostas.
    *   Explorar técnicas de clustering de clientes (se dados permitirem) para refinar a segmentação.
    *   Analisar a evolução temporal dos padrões.

---
## 6. Análise Comparativa: Solução LLM (Fase 2) vs. Solução do Grupo (Fase 3)

Nesta seção, comparamos a abordagem e os resultados obtidos com o auxílio direto do LLM na Fase 2 com os desenvolvimentos, refinamentos e análises adicionais realizadas pelo grupo na Fase 3.

**Abordagem e Resultados da Fase 2 (Auxílio do LLM):**

*   O LLM foi fundamental na estruturação do notebook da Fase 2, seguindo o CRISP-DM.
*   Gerou código funcional para carregamento, limpeza básica (nulos, tipos de dados, remoção de quantidades negativas e cancelamentos baseados em 'C' no TransactionNo, embora a remoção de cancelamentos não tenha sido efetiva inicialmente e precisou de ajuste manual no código da Fase 2) e transformação para formato transacional.
*   Sugeriu e implementou a modelagem com FP-Growth e a geração de regras de associação com os parâmetros definidos na proposta (`min_support=0.01`, `min_confidence=0.5`).
*   Produziu visualizações iniciais (scatter plot e um grafo básico com as top 50 regras por lift).
*   As interpretações fornecidas pelo LLM para as regras e gráficos foram genéricas, mas corretas em um nível superficial (ex: identificou o cluster dos Herb Markers).

**Desenvolvimentos e Refinamentos da Fase 3 (Grupo):**

*   **Consolidação da Limpeza:** O código de limpeza foi revisado e consolidado, garantindo que todas as etapas identificadas como necessárias na Fase 2 fossem aplicadas de forma robusta.
*   **Análise Segmentada por País:** Esta foi a principal adição do grupo na Fase 3, atendendo a um objetivo específico da proposta (TP1) que não foi proativamente abordado pelo LLM.
    *   Foram realizadas análises separadas para Alemanha e França.
    *   Isso exigiu a filtragem dos dados, re-transformação para formato transacional e, crucialmente, o **ajuste dos parâmetros `min_support` e `min_confidence`** para se adequarem ao volume de dados menor de cada segmento. O LLM na Fase 2 aplicou os parâmetros globais. O grupo utilizou uma heurística para ajustar o `min_support` dinamicamente e optou por um `min_confidence` menor para os segmentos.
    *   Essa análise segmentada revelou padrões de compra distintos em cada país, que seriam mascarados na análise geral.
*   **Aprofundamento da Avaliação:**
    *   A interpretação dos resultados foi significativamente aprofundada, comparando os padrões entre os diferentes segmentos e a análise geral.
    *   Foram propostas ações de negócio mais específicas e direcionadas para cada segmento, além das gerais.
    *   A avaliação dos critérios de sucesso foi feita de forma mais explícita e consolidada.
*   **Refinamento da Interpretação do Grafo:** A interpretação do grafo de rede foi elaborada com mais detalhes, identificando múltiplos clusters e suas implicações de negócio.
*   **Identificação de Limitações:** O grupo identificou limitações mais específicas da análise, como a questão do volume de dados para outros segmentos e a ausência de análise por tipo de cliente.

**Justificativas para as Mudanças/Adições:**

*   A **análise segmentada** foi realizada para cumprir um requisito explícito da Proposta TP1 e para obter insights mais granulares e acionáveis, reconhecendo que o comportamento do consumidor pode variar geograficamente.
*   O **ajuste de parâmetros** para os segmentos foi necessário porque aplicar os mesmos limiares do dataset completo a subconjuntos muito menores resultaria em poucas ou nenhuma regra significativa.
*   O **aprofundamento da avaliação e das propostas de ação** reflete a análise crítica humana, que vai além da geração de resultados e busca extrair valor de negócio e reconhecer as nuances dos dados.

**Aprendizados sobre o Uso do LLM:**

*   **Eficiência na Prototipagem:** O LLM é excelente para gerar rapidamente o esqueleto do projeto e o código para tarefas padrão, economizando tempo na fase inicial de desenvolvimento.
*   **Necessidade de Direcionamento Específico:** Para objetivos mais específicos ou análises mais detalhadas (como a segmentação ou o ajuste fino de parâmetros para subconjuntos), o LLM requer prompts muito precisos ou a intervenção humana para identificar e implementar essas etapas.
*   **Interpretação vs. Geração:** O LLM pode gerar interpretações básicas, mas a análise crítica, a conexão com o contexto de negócio específico e a identificação de insights mais profundos ainda são predominantemente tarefas humanas.
*   **Limitações Técnicas (Interação):** A dificuldade do LLM em editar diretamente o notebook foi um ponto de atrito, embora contornável.

**Conclusão da Análise Comparativa:**

A Fase 2, com auxílio do LLM, forneceu uma base sólida e acelerou o desenvolvimento inicial. A Fase 3, conduzida pelo grupo, construiu sobre essa base, adicionando análises mais complexas e específicas (segmentação), refinando a interpretação e a avaliação, e demonstrando a importância da análise crítica humana para extrair valor máximo dos resultados da mineração de dados. O LLM foi um bom "primeiro analista" ou "assistente de codificação", mas a profundidade e a especificidade da análise final foram contribuições do grupo.

---

## 7. Considerações Finais da Fase 3 e do Projeto

Este trabalho prático permitiu aplicar o processo de Mineração de Dados, desde a compreensão do negócio até a avaliação dos resultados, em um dataset real de e-commerce. O foco na mineração de padrões frequentes revelou associações interessantes entre produtos, tanto na análise geral quanto nas análises segmentadas por país (Alemanha e França).

**Principais Conquistas:**

*   Implementação bem-sucedida do fluxo CRISP-DM.
*   Limpeza e preparação eficazes dos dados.
*   Aplicação do algoritmo FP-Growth e geração de regras de associação significativas.
*   Identificação de padrões de compra distintos para o mercado geral (UK) e para segmentos específicos (Alemanha, França), levando a propostas de ações de negócio direcionadas.
*   Utilização e avaliação crítica de um LLM como ferramenta auxiliar no processo.

**Desafios e Aprendizados:**

*   A necessidade de ajustar os parâmetros de mineração (`min_support`, `min_confidence`) para diferentes volumes de dados (dataset completo vs. segmentos) foi um aprendizado importante.
*   A interpretação das regras e a tradução dos padrões em ações de negócio acionáveis requerem um bom entendimento do contexto e análise crítica.
*   A análise segmentada, embora mais trabalhosa, pode fornecer insights muito mais valiosos do que uma análise puramente agregada, especialmente em datasets com diversidade geográfica ou de clientes.

**Trabalhos Futuros (Além do Escopo deste TP):**

*   Explorar outros países com volume de dados suficiente.
*   Incorporar a dimensão temporal para identificar sazonalidades ou tendências nos padrões de compra.
*   Utilizar métricas adicionais para avaliação de regras.
*   Se dados permitirem, segmentar a análise por tipo de cliente.
*   Testar a implementação das ações de negócio propostas e medir seu impacto.

Este projeto cumpriu os objetivos definidos na proposta, fornecendo uma base sólida de insights sobre o comportamento de compra dos clientes e demonstrando o valor da mineração de regras de associação para o e-commerce.